<a href="https://colab.research.google.com/github/Maspie/Roadmap/blob/main/bias_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import important libraries

In [1]:
pip install fairlearn


In [2]:
import pandas as pd
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference


In [3]:
  df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data", header = None)

# df.head()

columns = ["age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
           "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
           "hours-per-week", "native-country", "income"]

df.columns = columns

df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
# Pre-processing
df = df.apply(lambda col: col.map(lambda x: x.lower() if isinstance(x, str) else x))

# Trimming Spaces

df = df.apply(lambda col: col.map(lambda x: x.strip() if isinstance(x, str) else x))

df["sex"] = df["sex"].map({"male": 1, "female": 0})

# print(df["race"].unique())

df["race"] = df["race"].map({"white": 1,
    "black": 2,
    "asian-pac-islander": 3,
    "amer-indian-eskimo": 4,
    "other": 5})

# print(df["income"].unique())

df["income"] = df["income"].map({"<=50k": 1, ">50k": 0})

df = df.drop(["fnlwgt", "education", "native-country"], axis=1)

# print(df.head())


bias_column = "race"
target_column = "income"


In [5]:

dem_parity = demographic_parity_difference(y_true=df[target_column], y_pred=df[target_column], sensitive_features=df[bias_column])
eq_odds = equalized_odds_difference(y_true=df[target_column], y_pred=df[target_column], sensitive_features=df[bias_column])


# Print results
print(f"Fairness Analysis for {bias_column}:")
print(f"Demographic Parity Difference: {dem_parity:.4f}")
print(f"Equalized Odds Difference: {eq_odds:.4f}")

# Check for significant bias
if abs(dem_parity) > 0.1 or abs(eq_odds) > 0.1:
    print("\n⚠️ Bias Detected!")
else:
    print("\n✅ No significant bias detected.")

Fairness Analysis for race:
Demographic Parity Difference: 0.1734
Equalized Odds Difference: 0.0000

⚠️ Bias Detected!


In [5]:
,jdfb